## ETAPA 1

#### Cargar datos ☑︎
#### Cargar modelos ☒
#### Entrenar modelos ☒
#### Ver resultados ☒

In [31]:
import glob
import shutil
import os
import re
import random

In [122]:
# BLOQUE 1: Separar data-set en carpetas

#file = open("Data-set/PA-expert-annotations.txt", "r")
#os.mkdir("Data-set/datos")
#os.mkdir("Data-set/datos/01-Normal")
#os.mkdir("Data-set/datos/02-Tapered")
#os.mkdir("Data-set/datos/03-Pyriform")
#os.mkdir("Data-set/datos/04-Small")
#os.mkdir("Data-set/datos/05-Amorphous")

#categorias = ["01-Normal", "02-Tapered", "03-Pyriform", "04-Small", "05-Amorphous"]

#for x in file:
#    # Se obtiene el nombre y clase desde el archivo txt
#    aux1 = x.split('	')
#    clase = int(aux1[4].replace('\n',''))
#    array.append(clase)
#    aux2 = aux1[0].replace('\n','').split('-')
#    p = aux2[0]
#    pl = aux2[1]
#    aux3 = aux2[2].split('/')
#    n_sample = int(re.split('(\d+)',aux3[0])[1])
#    n_sperm = int(re.split('(\d+)',aux3[1])[1])
#    # Se conforma el directorio de la imagen a partir de la informacion anterior.
#    file = 'Data-set/Partial-Agreement-Images/ch00_'+p+'-'+pl+'-sample'+str(n_sample)+'-sperm'+str(n_sperm)+'.tif'
#    # Se conforma el directorio donde se va a copiar la imagen
#    dest_dir = 'Data-set/data/'
#    if (clase == 0):
#        dest_dir=dest_dir+'01-Normal'
#    elif (clase == 1):
#        dest_dir=dest_dir+'02-Tapered'
#    elif (clase == 2):
#        dest_dir=dest_dir+'03-Pyriform'
#    elif (clase == 3):
#        dest_dir=dest_dir+'04-Small'
#    else:
#        dest_dir=dest_dir+'05-Amorphous'
#    
#    # Se copia la imagen
#    shutil.copy(file,dest_dir)

In [124]:
def leerClases( dir ):
    file = open(dir, "r")
    clases = []
    for x in file:
        # Se obtiene el nombre y clase desde el archivo txt
        aux1 = x.split('	')
        clase = int(aux1[4].replace('\n',''))
        clases.append(clase) 
    return clases


In [125]:
# BLOQUE 2: Crear data-set balanceado

# Se aplica la estrategia: Random subsampling en clase mayoritaria AMORFO

clases = leerClases("Data-set/PA-expert-annotations.txt")
print("Clase 0: "+str(clases.count(0))+"\n"+"Clase 1: "+str(clases.count(1))+
      "\n"+"Clase 2: "+str(clases.count(2))+"\n"+"Clase 3: "+str(clases.count(3))+
      "\n"+"Clase 4: "+str(clases.count(5))+"\n")

# Se obtiene promedio entre las clases minotarias ( Clase 0, 1, 2 y 3)
promedio = (clases.count(0)+clases.count(1)+clases.count(2)+clases.count(3))/4


lista = os.listdir('Data-set/b_data/05-Amorphous')

# Se eliminan aleatoriamentes elementos de la clase mayoritaria hasta llegar al promedio.
while len(lista) > promedio:
    elemento = random.choice(lista)
    lista.remove(elemento)
    elemento = 'Data-set/b_data/05-Amorphous/'+elemento
    os.remove( elemento )


Clase 0: 100
Clase 1: 228
Clase 2: 76
Clase 3: 72
Clase 4: 656



In [146]:
# Dividir Data-set original y balanceado.

def crearSubConjuntos( nombre_clase, dir_clase, dest_dir):
    os.mkdir(dest_dir+"/train/"+nombre_clase)
    os.mkdir(dest_dir+"/test/"+nombre_clase)
    os.mkdir(dest_dir+"/valid/"+nombre_clase)
    
    lista = os.listdir(dir_clase)
    contador = 0
    while (len(lista)*0.1) > contador:
        elemento = random.choice(lista)
        lista.remove(elemento)
        elemento = dir_clase+'/'+elemento
        shutil.copy(elemento,dest_dir+"/test/"+nombre_clase)
        contador = contador+1

    contador = 0
    while (len(lista)*0.1) > contador:
        elemento = random.choice(lista)
        lista.remove(elemento)
        elemento = dir_clase+'/'+elemento
        shutil.copy(elemento,dest_dir+"/valid/"+nombre_clase)
        contador = contador+1

    while len(lista) > 0:
        elemento = random.choice(lista)
        lista.remove(elemento)
        elemento = dir_clase+'/'+elemento
        shutil.copy(elemento,dest_dir+"/train/"+nombre_clase)
        contador = contador+1

def crearConjuntos( root_dir, dest_dir ):
    os.mkdir(dest_dir+"/train")
    os.mkdir(dest_dir+"/test")
    os.mkdir(dest_dir+"/valid")
    crearSubConjuntos( '01-Normal', root_dir+'/01-Normal', dest_dir )
    crearSubConjuntos( '02-Tapered', root_dir+'/02-Tapered', dest_dir )
    crearSubConjuntos( '03-Pyriform', root_dir+'/03-Pyriform', dest_dir )
    crearSubConjuntos( '04-Small', root_dir+'/04-Small', dest_dir )
    crearSubConjuntos( '05-Amorphous', root_dir+'/05-Amorphous', dest_dir )


In [147]:
#crearConjuntos('Data-set/data','Data-set/final_data' )
#crearConjuntos('Data-set/b_data','Data-set/final_b_data' )

In [124]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from PIL import Image

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img


dataGenerator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1/255)

test_it = dataGenerator.flow_from_directory('Data-set/final_b_data/test', class_mode='categorical',target_size=(34, 34))
valid_it = dataGenerator.flow_from_directory('Data-set/final_b_data/valid', class_mode='categorical',target_size=(34, 34))
train_it = dataGenerator.flow_from_directory('Data-set/final_b_data/train', class_mode='categorical',target_size=(34, 34))

img = load_img('/Users/matiascoronado/Desktop/Codigo/Data-set/data/03-Pyriform/ch00_p1-pl2-sample2-sperm15.tif')  # this is a PIL image
x = img_to_array(img)



Found 56 images belonging to 5 classes.
Found 51 images belonging to 5 classes.
Found 488 images belonging to 5 classes.


In [125]:
# BLOQUE3: Conformar red CNN

In [126]:
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Conv2D, AveragePooling2D, Concatenate, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, AveragePooling2D, GlobalMaxPooling2D, BatchNormalization, MaxPooling2D
from tensorflow.keras.models import Model

model = tf.keras.Sequential()
model.add(Conv2D(filters=64, kernel_size=(2, 2), input_shape=(34, 34, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(2, 2), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(GlobalMaxPooling2D())
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))
model.summary()



Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_52 (Conv2D)           (None, 33, 33, 64)        832       
_________________________________________________________________
max_pooling2d_39 (MaxPooling (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_53 (Conv2D)           (None, 16, 16, 128)       32896     
_________________________________________________________________
max_pooling2d_40 (MaxPooling (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_54 (Conv2D)           (None, 8, 8, 256)         131328    
_________________________________________________________________
max_pooling2d_41 (MaxPooling (None, 4, 4, 256)         0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 4, 4, 512)       

In [127]:
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.CategoricalCrossentropy(),
  metrics=[tf.keras.metrics.CategoricalCrossentropy(name="accuracy")])

In [129]:
size = 64
ep = 50
ver = 1

model_histort = model.fit_generator(
        train_it,
        steps_per_epoch=64,
        epochs=50,
        validation_data=test_it,
        validation_steps=50)

view_model_history(model_history)



Epoch 1/50
41/64 [==================>...........] - ETA: 6s - loss: 0.0625 - accuracy: 0.0634

KeyboardInterrupt: 

In [115]:
# AlexNet
# ResNet50
# EfficientNet B7

In [71]:
# BLOQUE4: Aplicar Grid

In [ ]:
# BLOQUE5: Aplicar RandAugment